#### This Notebook aims to test pyspark functions that handle missing value in a dataframe

In [27]:
# Testing pyspark installation
# you have to install findspark with this command: conda install -c conda-forge findspark
# Make sure if you are behind a corporate proxy to whitelist conda-forge channel
import findspark
findspark.init()
findspark.find()

import os
# Make sure you add the good version of the package hadoop-aws, compatible to hadoop version already installed
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.hadoop:hadoop-aws:3.2.0 --conf spark.dynamicAllocation.enabled=true pyspark-shell '

In [2]:
# Get AWS Credentials from local file 
import os
import configparser

config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))

aws_profile = 'default' # your AWS profile to use

access_key = config.get(aws_profile, "aws_access_key_id") 
secret_key = config.get(aws_profile, "aws_secret_access_key")

In [3]:
import pandas as pd

df = pd.read_csv("../data/reviews.csv")

df.head()


,listing_id,date
0,2818,2009-03-30
1,2818,2009-04-24
2,2818,2009-05-03
3,2818,2009-05-18
4,2818,2009-05-25


In [28]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Dataframe").getOrCreate()
spark


In [29]:
#pdf = spark.read.option("header", "true").csv("../data/reviews.csv")
#dfp.printSchema()

pdf = spark.read.option("header","true").csv('../data/reviews.csv',inferSchema=True)
pdf.show(5)

+----------+----------+
|listing_id|      date|
+----------+----------+
|      null|      null|
|      null|2009-04-24|
|      2818|2009-05-03|
|      2818|2009-05-18|
|      2818|2009-05-25|
+----------+----------+
only showing top 5 rows



In [10]:
## Drop columns with at least null value

pdf.na.drop(how="any").show(5)

+----------+----------+
|listing_id|      date|
+----------+----------+
|      2818|2009-04-24|
|      2818|2009-05-03|
|      2818|2009-05-18|
|      2818|2009-05-25|
|      2818|2009-06-29|
+----------+----------+
only showing top 5 rows



In [17]:
## Drop columns with threshold

pdf.na.drop(how="any", thresh=1).show(5)

+----------+----------+
|listing_id|      date|
+----------+----------+
|      2818|      null|
|      2818|2009-04-24|
|      2818|2009-05-03|
|      2818|2009-05-18|
|      2818|2009-05-25|
+----------+----------+
only showing top 5 rows



In [18]:
## Subset

pdf.na.drop(how="any", subset=['date']).show(5)

+----------+----------+
|listing_id|      date|
+----------+----------+
|      2818|2009-04-24|
|      2818|2009-05-03|
|      2818|2009-05-18|
|      2818|2009-05-25|
|      2818|2009-06-29|
+----------+----------+
only showing top 5 rows



In [21]:
pdf.na.fill('zero').show()

+----------+----------+
|listing_id|      date|
+----------+----------+
|      2818|      zero|
|      2818|2009-04-24|
|      2818|2009-05-03|
|      2818|2009-05-18|
|      2818|2009-05-25|
|      2818|2009-06-29|
|      2818|2009-07-07|
|      2818|2009-09-06|
|      2818|2009-10-01|
|      2818|      zero|
|      2818|2009-11-23|
|      2818|2009-11-25|
|      2818|2010-01-27|
|      2818|      zero|
|      2818|2010-04-16|
|      2818|2010-04-19|
|      2818|2010-05-03|
|      2818|2010-05-24|
|      2818|2010-06-01|
|      2818|2010-06-17|
+----------+----------+
only showing top 20 rows



In [30]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['listing_id'],
    outputCols= ["{}_imputed".format(c) for c in ['listing_id']]
).setStrategy("mean")

In [31]:
imputer.fit(pdf).transform(pdf).show()

+----------+----------+------------------+
|listing_id|      date|listing_id_imputed|
+----------+----------+------------------+
|      null|      null|          13869983|
|      null|2009-04-24|          13869983|
|      2818|2009-05-03|              2818|
|      2818|2009-05-18|              2818|
|      2818|2009-05-25|              2818|
|      2818|2009-06-29|              2818|
|      2818|2009-07-07|              2818|
|      2818|2009-09-06|              2818|
|      2818|2009-10-01|              2818|
|      2818|      null|              2818|
|      2818|2009-11-23|              2818|
|      2818|2009-11-25|              2818|
|      2818|2010-01-27|              2818|
|      2818|      null|              2818|
|      2818|2010-04-16|              2818|
|      2818|2010-04-19|              2818|
|      2818|2010-05-03|              2818|
|      2818|2010-05-24|              2818|
|      2818|2010-06-01|              2818|
|      2818|2010-06-17|              2818|
+----------